In [ ]:
# import and downloads all required packages 

In [ ]:
import nltk
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict 
import numpy as np
from statistics import variance
import nltk
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
# Convert required fields of json file in dataframe

In [ ]:
input_json_file = '/content/gdrive/MyDrive/nlp/Clickbit_dataset/clickbait17-validation-170630/instances.jsonl'

In [ ]:
def load_dataset(file_name):
    import pandas as pd
    import json
    df = []
    with open(file_name)as f:
        for i in f:
          i  = json.loads(i)
          id = i["id"]
          p  = i['targetParagraphs'] 
          t  = i["postText"]
          k  = i["targetKeywords"]
          tl = i["targetTitle"]          
          df += [{'id':id, 'p':p, 'title':tl, 'key':k, 'text':t }]
    return pd.DataFrame(df)   
          
dataset = load_dataset(input_json_file)

In [ ]:
dataset.head()

,id,p,title,key,text
0,858462320779026433,[Thousands of modern slavery victims have not ...,‘Inexcusable’ failures in UK’s response to mod...,"modern slavery, Department For Work And Pensio...",[UK’s response to modern slavery leaving victi...
1,858421020331560960,[President Donald Trump has appointed the pro-...,Donald Trump Appoints Pro-Life Advocate as Ass...,"Americans United for Life, Dr. Charmaine Yoest...",[this is good]
2,858368123753435136,[When the White House correspondents’ dinner i...,The ‘forgotten’ Trump roast: Relive his brutal...,"trump whcd, whcd, white house correspondents d...","[The ""forgotten"" Trump roast: Relive his bruta..."
3,858323428260139008,[Adorable is probably an understatement. This ...,"Meet The Happiest Dog In The World, Maru The H...","Maru, husky, dogs, pandas, furball, instagram",[Meet the happiest #dog in the world!]
4,858283602626347008,[One of Tokyo's major subways systems says it ...,Tokyo's subway is shut down amid fears over an...,"Tokyo,subway,shut,fears,North,Korean,attack",[Tokyo's subway is shut down amid fears over a...


In [ ]:
dataset["p"][0]

['Thousands of modern slavery victims have\xa0not come forward, while others who have chosen to report their enslavers have ended up destitute as a result of insufficient support, say\xa0MPs',
 '“Inexcusable” failures in the UK’s system for dealing with modern slavery are\xa0leaving victims reduced to destitution while their abusers go free because they are not adequately supported to testify against them, an alarming report has warned.',
 'Thousands of\xa0victims\xa0have not come forward, while others who have chosen to give evidence against their enslavers have ended up destitute as a result of insufficient support, according to\xa0a report published\xa0today by\xa0the Work and Pensions Committee.',
 'It is estimated there are between 10,000 and 13,000 victims of modern slavery in the UK, but the report\xa0warns that failings in the current mechanism mean\xa0that once they are identified, they have no automatic formal immigration status or rights.',
 'Weak and uncoordinated frontline

In [ ]:
# Preprocessed the texts 

In [ ]:
def remove_(t):
  l = []
  for s in t:
    l1 = []
    for item in s:
      item = item.replace('\xa0','')
      l1.append(item)
    l.append(l1)
  return l

In [ ]:
para = remove_(dataset['p'])

In [ ]:
para[0]

['Thousands of modern slavery victims havenot come forward, while others who have chosen to report their enslavers have ended up destitute as a result of insufficient support, sayMPs',
 '“Inexcusable” failures in the UK’s system for dealing with modern slavery areleaving victims reduced to destitution while their abusers go free because they are not adequately supported to testify against them, an alarming report has warned.',
 'Thousands ofvictimshave not come forward, while others who have chosen to give evidence against their enslavers have ended up destitute as a result of insufficient support, according toa report publishedtoday bythe Work and Pensions Committee.',
 'It is estimated there are between 10,000 and 13,000 victims of modern slavery in the UK, but the reportwarns that failings in the current mechanism meanthat once they are identified, they have no automatic formal immigration status or rights.',
 'Weak and uncoordinated frontline support means victims are often faced w

In [ ]:
text = dataset["text"]
title = dataset["title"]

In [ ]:
df = pd.DataFrame()
df["id"] = dataset["id"]

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Import and download all the required packages for using sentences BERT

In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=692a8d85feb1e144024a6bda89c23c3710d6bde26b1793dbe74944048c48c1f1
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer 
from sklearn.metrics.pairwise import cosine_similarity
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
# Selected only half sentences of the  paragraphs for constructing the correlation matrix 

In [ ]:
def count(l):
  cnt = 0
  if(len(l)<=1):
    cnt = len(l)
  else:
    cnt  = len(l)//2
  return cnt

In [ ]:
# Used sentence BERT encoder to encode list of sentences into array of integers.

In [ ]:
def BERT_and_cosine(l):
  arr0 = model.encode(l)
  val = cosine_similarity(arr0,arr0)
  return val

def Sim_matrix_using_BERT_and_cosine(l):
  val = BERT_and_cosine(l)
  print(val)
  print(val.shape)
  d = {}
  for i in range(len(l)):
    for j in range(len(l)):
      if(val[i][j] in d):
        l1 = d[val[i][j]]
        l1.append([i,j])
        d[val[i][j]] = l1
      else:
        d[val[i][j]] = [[i,j]]
  return d
  

In [ ]:
len(para[0])

28

In [ ]:
# Calculation for mean and variance of array

In [ ]:
def positive(a):
  for i in range(len(a)):
    if a[i]<0:
      a[i] = -a[i]
  return a
def Flatten(arr):
  arr1 = arr[:][0]
  a = arr1.flatten()
  a = positive(a)
  return a
def mean_var(a):
  return  np.mean(a), np.var(a)

In [ ]:
# Prepared a list of posts texts, title and paragraphs sentences.

In [ ]:
def title_postText_para_i(Title,pText,para, i):
  l = []
  l.append(pText[i][0])
  l.append(Title[i])
  l.extend(para[i])
  # print(l)
  return l

In [ ]:
# Calculate the mean variance of the title and post texts corresponding to correlations matrix 

In [ ]:
from pandas.core.frame import DataFrame
def mean_var_list(title, text, paragraphs):
  p_mean = []
  p_var =  []
  t_mean = []
  t_var =  []
  # df1 = pd.DataFrame()
  for i in range(len(title)):
    l = title_postText_para_i(title, text, paragraphs,i)
    # print(len(l))
    arr0 = model.encode(l)
    arr = cosine_similarity(arr0,arr0)
    # a = Flatten(arr)
    m1,v1 = mean_var(arr[0])
    p_mean.append(m1)
    p_var.append(v1)
    m2 ,v2 = mean_var(arr[1])
    t_mean.append(m2)
    t_var.append(v2)
  df["postText mean"] = pd.DataFrame(p_mean)
  df["postText variance"] = pd.DataFrame(p_var)
  df["title mean"] = pd.DataFrame(t_mean)
  df["title variance"] = pd.DataFrame(t_var)
  # return df1

mean_var_list(title, text, para)


In [ ]:
df

,id,postText mean,postText variance,title mean,title variance
0,858462320779026433,0.643028,0.016918,0.678807,0.020574
1,858421020331560960,0.259717,0.070382,0.428651,0.059395
2,858368123753435136,0.619952,0.026573,0.631976,0.024935
3,858323428260139008,0.409056,0.056876,0.361099,0.105833
4,858283602626347008,0.596690,0.029145,0.596690,0.029145
...,...,...,...,...,...
19533,804250183642976256,0.542422,0.192908,0.540422,0.154097
19534,804156272086020096,0.201753,0.109962,0.386814,0.098140
19535,804149798651588608,0.586110,0.041550,0.594503,0.039105
19536,804134698729385984,0.335589,0.054357,0.567416,0.066583


In [ ]:
# Save features in csv files

In [ ]:
df.to_csv("/content/gdrive/MyDrive/nlp/Clickbit_dataset/clickbait17-validation-170630/feature/csv/mean_variance.csv",index=False)